In [1]:
import pandas as pd
import json,glob, os
import re
from datetime import datetime
import matplotlib.pyplot as plt
#from wordcloud import WordCloud, STOPWORDS
import requests
import os
# import aiohttp
import asyncio

In [2]:

json_dir = '../data/'

json_pattern = os.path.join(json_dir, '*_pr_sharings.json')
file_list = glob.glob(json_pattern)

dfs = []
for file in file_list:
	with open(file) as f:
		json_data = pd.json_normalize(json.loads(f.read()))
		json_data['site'] = file.rsplit("/", 1)[-1]
	dfs.append(json_data)
df = pd.concat(dfs)

In [3]:
df.head(10)

,Sources,site
0,"[{'Type': 'pull request', 'URL': 'https://gith...",20230831_060603_pr_sharings.json
0,"[{'Type': 'pull request', 'URL': 'https://gith...",20230907_091631_pr_sharings.json
0,"[{'Type': 'pull request', 'URL': 'https://gith...",20230817_125147_pr_sharings.json
0,"[{'Type': 'pull request', 'URL': 'https://gith...",20230824_100450_pr_sharings.json
0,"[{'Type': 'pull request', 'URL': 'https://gith...",20230727_195927_pr_sharings.json
0,"[{'Type': 'pull request', 'URL': 'https://gith...",20230803_093947_pr_sharings.json
0,"[{'Type': 'pull request', 'URL': 'https://gith...",20231012_233628_pr_sharings.json
0,"[{'Type': 'pull request', 'URL': 'https://gith...",20230810_123110_pr_sharings.json
0,"[{'Type': 'pull request', 'URL': 'https://gith...",20230914_074826_pr_sharings.json
0,"[{'Type': 'pull request', 'URL': 'https://gith...",02_18_2024_manual_pr_sharings.json


In [4]:
# Filter bug-fixes pull request 
# BUGFIX_KEYWORDS = ['failur', 'fail', 'npe ', ' npe', 'issue', 'except', 'broken',
#                    'crash', 'bug', 'differential testing', 'error', 'incorrect', 'flaw',
#                    'addresssanitizer', 'hang ', ' hang', 'jsbugmon', 'leak', 'permaorange',
#                    'random orange', 'intermittent', 'regression', 'test fix', 'problem',
#                    'heap overflow', 'exception', 'daemon', 'stopped', 'broken', ' fault', 'race condition',
#                    'deadlock', 'synchronization error', 'dangling pointer', 'null pointer', 'overflow', 'memory leak',
#                    'race condition', 'restart', 'steps to reproduce', 'crash', 'assertion', 'failure', 'leak',
#                    'stack trace', 'defect', 'mistake', 'fix', 'avoid',
#                    'regression', 'test fix', ' hang', 'hang ', 'heap overflow', 'mozregression', 'safemode',
#                    'safe mode', 'stop'
#                    ]
def unique(list):
    unique_list = pd.Series(list).drop_duplicates().to_list()
    return unique_list

# bugs = unique(BUGFIX_KEYWORDS)

In [5]:
# total number of pull requests
len(df['Sources'])
total = [len(x) for x in df['Sources']]
sum = 0
for element in total:
    sum = sum + element
print(sum)        

1775


In [6]:
# get merged and/or closed PR

pull_request_merged = []
pull_request_closed = []

for item in df['Sources']:
    for i in item:
        if i['State'] == 'MERGED':
            pull_request_merged.append(i['URL'])
        if i['State'] == 'CLOSED':
            pull_request_closed.append(i['URL'])


In [7]:
len(pull_request_merged)

1363

In [8]:
len(pull_request_closed)

233

In [9]:
# We are interested in the merged PRs. Let create list of projects
# from the merged PR lists
items = []
for pr in pull_request_merged:
    items.append(pr.split('/pull/')[0])
projects = unique(items)

In [10]:
len(projects)

231

In [11]:
# helper function
"""
    Read from list
"""
# token_file = 'tokens.txt'
token_file = '../../../tokens.txt'

token_list = []
with open(token_file, 'r') as f:
    for line in f.readlines():
        token_list.append(line.strip('\n'))

"""
    Manually add in notebook
"""
#token_list = ["nadHJok13mdkaamda", "ajd1819kdamda","..."]

lenTokens = len(token_list)

def get_response(url, token_list, ct):
    json_data = None

    # token_list, len_tokens = tokens()
    len_tokens = len(token_list)
    try:
        ct = ct % len_tokens
        headers = {'Authorization': 'Bearer {}'.format(token_list[ct])}
        request = requests.get(url, headers=headers)
        json_data = json.loads(request.content)
        ct += 1
    except Exception as e:
        print(e)
        print("Error in func: [get_response]...")
    return json_data, ct


In [12]:
def api_request(url, token):
    header = {'Authorization': 'token %s' % token}
    response = requests.get(url, headers=header)
    try:
        json_response = json.loads(response.content)
        return json_response
    except Exception as e:
        return response

In [13]:
#Filter `toy projects based on the following criteria
#---- 1. At least 2 developers ---
# 2. At least 100 commits
# 3. At least 1 review - this is pracific to pull request.
# two_developer = []
project_filter = []
for project in projects:
    # constract api url
    part = project.split('github.com/')
    # url = f'{part[0]}api.github.com/repos/{part[1]}/contributors'
    # try:
    #     request = api_request(url, token_list[0])
    #     # at least 2 developers
    #     if len(request) > 1:
    #         two_developer.append(project)

    # at least 100 commits
    try:
        commits_url = f'{part[0]}api.github.com/repos/{part[1]}/commits?per_page=100'
        fetch_commits = api_request(commits_url, token_list[0])
        if len(fetch_commits) >= 100:
            project_filter.append(project)
    except Exception as e:
        print("skipping...", e)
    # except Exception as e:
    #     print(project, e)

In [14]:
len(project_filter)

187

In [15]:

# async def fetch_comments(session, url, token):
#     headers = {"Authorization": f"Bearer {token}"}
#     async with session.get(url, headers=headers) as response:
#         return await response.json()

# async def main():
#     project_filter_set = set(project_filter)
#     pull_request_merged_set = set(pull_request_merged)
#     pull_request_clean = []

#     async with aiohttp.ClientSession() as session:
#         tasks = []

#         for pr in pull_request_merged_set:
#             pr_part = pr.split('/pull/')
#             if pr_part[0] in project_filter_set:
#                 pf_part = pr_part[0].split('github.com/')
#                 try:
#                     comments_url = f"{pf_part[0]}api.github.com/repos/{pf_part[1]}/pulls/{pr_part[1]}/reviews"
#                     tasks.append(fetch_comments(session, comments_url, token_list[0]))
#                 except Exception as e:
#                     print("skipping...", e)

#         responses = await asyncio.gather(*tasks)

#         for pr, response in zip(pull_request_merged_set, responses):
#             if len(response) >= 1:
#                 pull_request_clean.append(pr)

#     print(len(pull_request_clean))

# # Run the asyncio event loop
# asyncio.run(main())

In [16]:
# get the merged PRs that are part of the remainings project and perform 
# PR specific filtering... i.e. PR should have at least 1 review
# e.g url: https://api.github.com/repos/apache/kafka/pulls/14540/comments

pull_request_clean = []
for pf in project_filter:
    for pr in pull_request_merged:
        pr_part = pr.split('/pull/')
        if pf == pr_part[0]:
            # at least 1 reviews
            pf_part = pf.split('github.com/')
            try:
                comments_url = f"{part[0]}api.github.com/repos/{pf_part[1]}/pulls/{pr_part[1]}/reviews"
                fetch_comments = api_request(comments_url, token_list[0])
                if len(fetch_comments) >= 1:
                    pull_request_clean.append(pr)
            except Exception as e:
                print("skipping...", e)


In [17]:
# total number of PRs that met the selection criteria 
pull_request_clean = unique(pull_request_clean)
len(pull_request_clean)

208

In [18]:
len(pull_request_clean)

208

In [19]:
# for p in pull_request_clean:
#     print(p.split('.com/')[1].split('/pull')[0])

In [20]:
items = []
for pr in pull_request_clean:
    items.append(pr.split('/pull/')[0])
projects_clean = unique(items)
len(projects_clean)

166

In [21]:
prompt_result = []
pull_duration = []
chatgapt_link_404 = []
no_listofcode = []
for element in df['Sources']:
    for i in element:
        # pull_title = i['Title'].lower().replace(';', '-').replace(',', '-')
        # # for bug in bugs:
        # if any([bug in pull_title for bug in bugs]):
        if i['URL'] in pull_request_clean and i['State'] == 'MERGED':
            # print(i['State'])
            # if i['State'] == 'MERGED':
            # pull_request_merged.append(i['URL'])
            mergedAt = datetime.strptime(str(i['MergedAt']), '%Y-%m-%dT%H:%M:%SZ')
            createdAt = datetime.strptime(str(i['CreatedAt']), '%Y-%m-%dT%H:%M:%SZ')
            pr_duration = (mergedAt - createdAt).days
            pull_duration.append(pr_duration)
            try:
                count = 1
                if i['ChatgptSharing']:
                    #print(i['URL'])
                    try:
                        conversation = i['ChatgptSharing'][0]['Conversations']
                    except:
                        chatgapt_link_404.append(i['URL'])
            
                    for prompts in i['ChatgptSharing'][0]['Conversations']:
                        prompt_result.append(prompts['Prompt'])
                        try:
                            code = prompts['ListOfCode']
                            if len(code) == 0:
                                no_listofcode.append(i['URL'])
                        except:
                            no_listofcode.append(i['URL'])

                        for content in prompts['ListOfCode']:
                            if content['Content']:
                                # print(content['Content'])
                                repo_name = i['RepoName'] 
                                storage_dir = f'RepoData/{repo_name}/{i["Number"]}/chatgpt/'
                                if not os.path.exists(storage_dir):
                                    os.makedirs(storage_dir)
                                    path = f'{storage_dir}patch-{count}.patch'
                                    with open(path, 'x') as patch:
                                        patch.writelines(content['Content'])
                                else: 
                                    count = count + 1
                                    path = f'{storage_dir}patch-{count}.patch'
                                    with open(path, 'x') as patch:
                                        patch.writelines(content['Content'])

            except Exception as e:
                print("Skipping.... ", e, i['URL'])


Skipping....  'Conversations' https://github.com/SWM14-Architect/moview-core-service/pull/12
Skipping....  'Conversations' https://github.com/KudoAI/chatgpt.js/pull/92
Skipping....  'Conversations' https://github.com/daydule/frontend/pull/35
Skipping....  'Conversations' https://github.com/openai/evals/pull/771
Skipping....  'Conversations' https://github.com/decred/dcrdex/pull/2333
Skipping....  'Conversations' https://github.com/mindee/doctr/pull/1205
Skipping....  [Errno 17] File exists: 'RepoData/open-learning-exchange/myplanet/2230/chatgpt/patch-2.patch' https://github.com/open-learning-exchange/myplanet/pull/2230
Skipping....  [Errno 17] File exists: 'RepoData/QRTaxi/QRTaxi_Web/29/chatgpt/patch-2.patch' https://github.com/QRTaxi/QRTaxi_Web/pull/29
Skipping....  [Errno 17] File exists: 'RepoData/QRTaxi/QRTaxi_Web/26/chatgpt/patch-2.patch' https://github.com/QRTaxi/QRTaxi_Web/pull/26
Skipping....  [Errno 17] File exists: 'RepoData/app-sre/qontract-reconcile/3630/chatgpt/patch-2.pat

In [22]:
len(chatgapt_link_404)

56

In [23]:
no_listofcode = unique(no_listofcode)
len(no_listofcode)

111

In [24]:
def fetch_pullrequest_data(pullrequests, skip_pr, token_list, ct):
    token_length = len(token_list)
    
    for pullrequest in pullrequests:
        # pull request to skip
        if pullrequest in skip_pr:
            print(pullrequest)
            continue
        repo = pullrequest.split('https://github.com/')[1].split('/pull/')
        try:
            
            # get files and patches
            if ct == token_length:
                ct = 0
            #files_merged = f'{constant.GITHUB_API_BASE_URL}{mainline}/commits/{merge_commit_sha}'
            files_merged = f'https://api.github.com/repos/{repo[0]}/pulls/{repo[1]}/files?page=1&per_page=100'
            pullrequest_files_merged, ct = get_response(files_merged, token_list, ct)
            #print(pullrequest_files_merged)
            ct += 1
        
            try:
                count = 1
                for file in  pullrequest_files_merged:
                    try:
                        patch = file['patch']
                        storage_dir = f'RepoData/{repo[0]}/{repo[1]}/github/'
                        if not os.path.exists(storage_dir):
                            os.makedirs(storage_dir)
                            path = f'{storage_dir}patch-{count}.patch'
                            with open(path, 'x') as file:
                                file.writelines(patch)
                        else: 
                            count = count + 1
                            path = f'{storage_dir}patch-{count}.patch'
                            with open(path, 'x') as file:
                                file.writelines(patch)
                    except Exception as e:
                        print("Skipping this patch...")
                    
            except Exception as e:
                print(e, pullrequest)
        except Exception as e:
            print("Error while trying to fetch pull request data....: ", pullrequest)

    return 1

In [25]:
fetch_pullrequest_data(pullrequests=pull_request_clean, skip_pr=chatgapt_link_404, token_list=token_list, ct=0)

Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...


Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping this patch...
Skipping th

1

In [26]:
path  = 'RepoData/'
repoData = [repo for repo in os.listdir(path) if not repo.startswith('.')]
repoData[0:3]

['Hochfrequenz', 'kube-hetzner', 'pyspark-ai']

In [27]:
len(repoData)

151

In [28]:
# sum = 0
# for v in pull_duration:
#     sum = sum + v
# print("Min: ", min(pull_duration), " Max: ", max(pull_duration), " Avg: ",  sum/len(pull_duration)) 

In [29]:
# len(pull_duration)

In [30]:
# # box plot
# fig = plt.figure(figsize=(4,4)) # dpi=1200
# plt.boxplot(pull_duration)
# plt.show()
# plt.close()

In [31]:
# violin plot
# fig = plt.figure(figsize=(4,4)) #dpi=1200
# plt.violinplot(pull_duration, showmeans=True, showextrema=False)
# plt.show()
# plt.close()

In [32]:

# unique_string = (" ").join(prompt_result).lower()
# wordcloud = WordCloud(
#     width=1000,
#     height=500,
#     background_color='white',
#     stopwords=STOPWORDS
#     )
# example = wordcloud.generate(unique_string)
# plt.figure(figsize=(10,4)) #dpi=1200
# plt.imshow(wordcloud)
# plt.axis("off")
# #plt.savefig("your_file_name"+".png", bbox_inches='tight')
# plt.show()
# plt.close()

In [33]:
# print(len(example.words_))